# Практическая часть к лекции № 1 "Сериализация и сохранение моделей. Pipeline."
## 1. Чтение данных

В качестве рабочего датасета возьмём классический "Титаник".

In [77]:
import numpy as np
import re

import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

np.random.seed(0)

In [78]:
data = pd.read_csv(re.sub('\s+', '', """https://raw.githubusercontent.com
                           /MariaZharova/Python-for-Data-Analysis/main/Pandas/train.csv"""))
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [71]:
display(data.describe())
data.astype('object').describe(include=['object'])

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891,891,891,891,891,714.0,891,891,891,891.00,204,889
unique,891,2,3,891,2,88.0,7,7,681,248.00,147,3
top,1,0,3,"Braund, Mr. Owen Harris",male,24.0,0,0,347082,8.05,B96 B98,S
freq,1,549,491,1,577,30.0,608,678,7,43.00,4,644


In [72]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


## 2. Pipeline
Это шаблон-схема для обработки данных

In [73]:
# неинформативные столбцы
data.drop(['Name', 'PassengerId', 'Ticket', 'Cabin'], inplace=True, axis=1)

# выделяем признаки и таргет
X = data.drop('Survived', axis=1)
y = data['Survived']

# числовые фичи
numeric_features = ["Age", "Fare"]
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)

# категориальные фичи
categorical_features = ["Embarked", "Sex", "Pclass"]
categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore")),
        ("selector", SelectPercentile(chi2, percentile=50)),
    ]
)

# для последующей предобработки фичей разного типа
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

In [74]:
# соединяем предобработку и обучение в Pipeline
clf = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LogisticRegression())]
)

# дальше - дело техники
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

clf.fit(X_train, y_train)
print("model score: %.3f" % clf.score(X_test, y_test))

model score: 0.799


In [37]:
# попробуйте запустить 
# clf

![title](1.png)

## 3. Сохранение моделей: pickle и joblib

In [38]:
import pickle
import joblib

In [ ]:
# проверяем, что модель не пустая
clf['classifier']
# clf['classifier'].coef_

In [45]:
# сохранение модели (сериализация)
with open('my_model.pkl', 'wb') as output:
    pickle.dump(clf['classifier'], output)

In [46]:
# загрузка из файла (десериализация)
with open('my_model.pkl', 'rb') as pkl_file:
    model_from_file = pickle.load(pkl_file)

model_from_file

LogisticRegression()

In [51]:
# сравним адреса моделей в памяти
print(id(clf['classifier']))
print(id(model_from_file))

140266652425456
140266652966560


In [49]:
# что не так с этой строкой?
all(clf['classifier'].predict(X_test) == model_from_file.predict(X_test))

##### Поэтому, сохраним весь пайплайн!

In [52]:
# сохранение пайплайна (сериализация)
with open('pipe.pkl', 'wb') as output:
    pickle.dump(clf, output)

In [54]:
# загрузка из пайплайна (десериализация)
with open('pipe.pkl', 'rb') as pkl_file:
    pipeline_from_file = pickle.load(pkl_file)

In [55]:
# Теперь проверим, что все элементы массивов предсказаний совпадают между собой
all(clf.predict(X_test) == pipeline_from_file.predict(X_test))

True

##### А ещё есть билиотека joblib

In [56]:
# аналогично для joblib
joblib.dump(clf, 'pipe.joblib')

pipeline_from_joblib = joblib.load('pipe.joblib') 

# Сравниваем предсказания
all(clf.predict(X_test) == pipeline_from_joblib.predict(X_test))

True

In [57]:
print(id(clf))
print(id(pipeline_from_joblib))

140266652427136
140266652433376


##### А что дальше?
Сохранив пайплайн, можно делать инференс модели!

##### Иногда бывает необходимо построить кастомный пайплайн (например, для создания новой фичи)
Для этого нам понадобится "кастомный трансформер". Вот пример шаблона:

In [ ]:
class MyTransformer(TransformerMixin, BaseEstimator):
    '''Шаблон кастомного трансформера'''


    def __init__(self):
        '''Здесь прописывается инициализация параметров, не зависящих от данных.'''
        pass


    def fit(self, X, y=None):
        '''
        Здесь прописывается «обучение» трансформера.
        Вычисляются необходимые для работы трансформера параметры (если они нужны).
        '''
        return self


    def transform(self, X):
        '''Здесь прописываются действия с данными.'''
        # Создаём новый столбец как произведение первых трёх
        new_column = X[:, 0] * X[:, 1] * X[:, 2]
        # Для добавления столбца в массив нужно изменить его размер на (n_rows, 1)
        new_column = new_column.reshape(X.shape[0], 1)
        # Добавляем столбец в матрицу измерений
        X = np.append(X, new_column, axis=1)
        return X

**Больше примеров для Pipeline:** https://gist.github.com/amberjrivera/8c5c145516f5a2e894681e16a8095b5c

**Официальная документация:** https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

## 4. Сохранение моделей:  PMML
Эта библиотека (а также ONNX) используются для сохранения нейросетевых моделей и если необходимо в дальнейшем встраивать их в сервисы на других языках, отличных от Python.

📌 Может сохранять только простые трансформеры в пайплайнах!

In [66]:
# ! pip install nyoka

In [76]:
from nyoka import skl_to_pmml
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_diabetes

X, y = load_diabetes(return_X_y=True)
cols = load_diabetes()['feature_names']

scaler = MinMaxScaler()
pipe = Pipeline([  
            ('Scaling', MinMaxScaler()),
            ('Linear', LinearRegression())
        ])
# Обучение пайплайна, включающего линейную модель и нормализацию признаков
pipe.fit(X, y)
# Сохраним пайплайн в формате pmml в файл pipeline.pmml
skl_to_pmml(pipeline=pipe, col_names=cols, pmml_f_name="pipeline.pmml")